# Create Dataset

This will run data through a few pipelines.

In [3]:
import constants

In [4]:
import tensorflow as tf

import magenta

from magenta.pipelines import pipeline
from magenta.protobuf import music_pb2
from magenta.protobuf import generator_pb2
from magenta.pipelines import pipeline
from magenta.pipelines import pipelines_common
from magenta.pipelines import dag_pipeline
from magenta.pipelines import note_sequence_pipelines

import arrangement_create_dataset
import arrangement_model

import os

**POINTERS**
* `perf_extractor` converts a `NoteSequence` of MIDI-like sequence events (`NOTE ON`, `NOTE OFF`) to a `music.Performance` additionally encoding TIME SHIFT events.
* `encoder_pipeline` takes a list-like sequence of events and returns a `tf.train.SequenceExample` containing inputs and labels.
    * **TO-DO**: Write a new class like `OneHotEventSequenceEncoderDecoder` (important methods: `events_to_input` and `events_to_label`), need to overwrite the inherited `encode`

### Method Definitions

In [5]:
class EncoderPipeline(pipeline.Pipeline):
    """A Pipeline that converts performances to a model specific encoding."""

    def __init__(self, config, name):
        """Constructs an EncoderPipeline.

        Args:
          config: A PerformanceRnnConfig that specifies the encoder/decoder and
              note density conditioning behavior.
          name: A unique pipeline name.
        """
        super(EncoderPipeline, self).__init__(
            input_type=magenta.music.Performance,
            output_type=tf.train.SequenceExample,
            name=name)
        self._encoder_decoder = config.encoder_decoder
        self._control_signals = config.control_signals
        self._optional_conditioning = config.optional_conditioning

    def transform(self, performance):
        if self._control_signals:
            # Encode conditional on control signals.
            control_sequences = []
            for control in self._control_signals:
                control_sequences.append(control.extract(performance))
            control_sequence = zip(*control_sequences)
            if self._optional_conditioning:
                # Create two copies, one with and one without conditioning.
                encoded = [
                    self._encoder_decoder.encode(
                        zip([disable] * len(control_sequence), control_sequence),
                        performance)
                    for disable in [False, True]]
            else:
                encoded = [self._encoder_decoder.encode(
                    control_sequence, performance)]
        else:
            # Encode unconditional.
            encoded = [self._encoder_decoder.encode(performance)]
        return encoded

In [6]:
class PerformanceExtractor(pipeline.Pipeline):
    """Extracts polyphonic tracks from a quantized NoteSequence."""

    def __init__(self, num_velocity_bins, name=None):
        super(PerformanceExtractor, self).__init__(
            input_type=music_pb2.NoteSequence,
            output_type=magenta.music.Performance,
            name=name)
        self._num_velocity_bins = num_velocity_bins

    def transform(self, quantized_sequence):
        performances, stats = magenta.music.extract_performances(
            quantized_sequence,
            num_velocity_bins=self._num_velocity_bins)
        self._set_stats(stats)
        return performances

In [88]:
def build_pipeline_graph(config, 
                         eval_ratio, 
                         test_ratio,
                         collection_name):
    """Returns the Pipeline instance which creates the RNN dataset.

    Args:
        config: A PerformanceRnnConfig.
        eval_ratio: Fraction of input to set aside for evaluation set.
        test_ratio: Fraction of input to set aside for test set.

    Returns:
        A pipeline.Pipeline instance.
    """
    
    # Stretch by -5%, -2.5%, 0%, 2.5%, and 5%.
    stretch_factors = [0.95, 0.975, 1.0, 1.025, 1.05]

    # Transpose no more than a major third.
    transposition_range = range(-3, 4)

    partitioner = pipelines_common.RandomPartition(
        music_pb2.NoteSequence,
        ['eval_arrangement' + '_' + collection_name, 
         'test_arrangement' + '_' + collection_name, 
         'train_arrangement' + '_' + collection_name],
        [eval_ratio, test_ratio])
    dag = {partitioner: dag_pipeline.DagInput(music_pb2.NoteSequence)}

    for mode in ['eval', 'test', 'train']:
        key = mode + '_arrangement' + '_' + collection_name
        
        sustain_pipeline = note_sequence_pipelines.SustainPipeline(
            name='SustainPipeline_' + key)
        
        stretch_pipeline = note_sequence_pipelines.StretchPipeline(
            stretch_factors if mode == 'train' else [1.0],
            name='StretchPipeline_' + key)
        
        splitter = note_sequence_pipelines.Splitter(
            hop_size_seconds=30.0, name='Splitter_' + key)
        
        quantizer = note_sequence_pipelines.Quantizer(
            steps_per_second=config.steps_per_second, name='Quantizer_' + key)
        
        transposition_pipeline = note_sequence_pipelines.TranspositionPipeline(
            transposition_range if mode == 'train' else [0],
            name='TranspositionPipeline_' + key)
        
        perf_extractor = PerformanceExtractor(
            num_velocity_bins=config.num_velocity_bins,
            name='PerformanceExtractor_' + key)
            # input_type = music_pb2.NoteSequence
            # output_type = magenta.music.Performance
        
        encoder_pipeline = EncoderPipeline(
            config, name='EncoderPipeline_' + key)
            # input_type = magenta.music.Performance
            # output_type = tf.train.SequenceExample
    
        # Partition > Quantize > Extract Performance > Encode > Out
        dag[quantizer] = partitioner[key]
        dag[perf_extractor] = quantizer
        dag[encoder_pipeline] = perf_extractor
        dag[dag_pipeline.DagOutput(key)] = encoder_pipeline

    return dag_pipeline.DAGPipeline(dag)

In [89]:
def create_dataset():
    for collection_name in ['inputs', 'targets']:
        
        src_file = os.path.join(constants.COLLATED_NOTE_SEQ_DIR, 
                                   collection_name + '.tfrecord')
        output_dir = os.path.join(constants.ENCODED_SEQ_EXMPL_DIR)
        
        # Construct the pipeline graph
        pipeline_graph = build_pipeline_graph(
            eval_ratio = constants.EVAL_RATIO,
            test_ratio = constants.TEST_RATIO,
            collection_name = collection_name,
            config = constants.default_configs[constants.CONFIG])

        # Runs pipeline graph on a data source and writes output to dir
        pipeline.run_pipeline_serial(
            pipeline_graph,
            pipeline.tf_record_iterator(src_file, 
                                        pipeline_graph.input_type),
            output_dir)

In [90]:
# Defaults to 0, all logs are shown. 
# 1 to filter out INFO logs
# 2 to additionall filter out WARNING
# 3 to additionally filter out ERROR
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [92]:
create_dataset()

INFO:tensorflow:

Completed.

INFO:tensorflow:Processed 76 inputs total. Produced 76 outputs.
INFO:tensorflow:DAGPipeline_PerformanceExtractor_eval_arrangement_inputs_performance_lengths_in_seconds:
  [10,20): 2
  [30,40): 2
  [60,120): 2
INFO:tensorflow:DAGPipeline_PerformanceExtractor_eval_arrangement_inputs_performances_discarded_more_than_1_program: 0
INFO:tensorflow:DAGPipeline_PerformanceExtractor_eval_arrangement_inputs_performances_discarded_too_short: 0
INFO:tensorflow:DAGPipeline_PerformanceExtractor_eval_arrangement_inputs_performances_truncated: 0
INFO:tensorflow:DAGPipeline_PerformanceExtractor_eval_arrangement_inputs_performances_truncated_timewise: 0
INFO:tensorflow:DAGPipeline_PerformanceExtractor_test_arrangement_inputs_performance_lengths_in_seconds:
  [10,20): 1
  [20,30): 1
  [30,40): 5
  [60,120): 2
INFO:tensorflow:DAGPipeline_PerformanceExtractor_test_arrangement_inputs_performances_discarded_more_than_1_program: 0
INFO:tensorflow:DAGPipeline_PerformanceExtractor_

### Helpful IO

##### Read `.tfrecord` of NoteSequences.

In [118]:
tfrecord_path = './data/note_seqs/inputs.tfrecord'

In [120]:
records = list()
for record in pipeline.tf_record_iterator(tfrecord_path, music_pb2.NoteSequence):
    records.append(record)

# Hint: To look at first record, run `records[0]`
# Hint 2: This is useful to drill inside: records[0].feature_lists.feature_list['inputs'].feature

##### Instantiate pipelines

In [121]:
from magenta.models.performance_rnn import performance_model
from magenta.pipelines import note_sequence_pipelines

config = constants.default_configs['performance']

quantizer_instance = note_sequence_pipelines.Quantizer(steps_per_second = config.steps_per_second,
                                                       name='Quantizer_jupyter')
perf_extractor_instance = PerformanceExtractor(num_velocity_bins = config.num_velocity_bins)
encoder_pipeline_instance = EncoderPipeline(config,
                                            name='EncoderPipeline_jupyter')

##### Step-through pipelines

In [184]:
for record in pipeline.tf_record_iterator(tfrecord_path, music_pb2.NoteSequence):
    note_sequence1 = record
# note_sequence1

In [180]:
note_sequence2 = quantizer_instance.transform(note_sequence1)[0]

In [189]:
note_sequence3 = perf_extractor_instance.transform(note_sequence2)[0]

# note_sequence3.to_sequence()        # converts Performance to NoteSequence proto
# note_sequence3.__getitem__(4)       # return event at position
# note_sequence3.steps_per_second     # if Performance(BasePerformance)
# note_sequence3.steps_per_quarter    # if MetricPerformance(BasePerformance)
note_sequence3._events              # list of all events
# len(note_sequence3._events)         # len of events
# note_sequence3.num_steps            # len of sequence in quantized steps
# note_sequence3.steps                # list of the time step at each event in the sequence

# returns an iterator
# for i, event in enumerate(note_sequence3.__iter__()): 
#     print(event)
#     if i > 25:
#         break

[PerformanceEvent(1, 53),
 PerformanceEvent(1, 58),
 PerformanceEvent(1, 62),
 PerformanceEvent(1, 77),
 PerformanceEvent(3, 75),
 PerformanceEvent(2, 53),
 PerformanceEvent(2, 58),
 PerformanceEvent(2, 62),
 PerformanceEvent(1, 53),
 PerformanceEvent(1, 57),
 PerformanceEvent(1, 60),
 PerformanceEvent(3, 25),
 PerformanceEvent(2, 77),
 PerformanceEvent(2, 53),
 PerformanceEvent(2, 57),
 PerformanceEvent(2, 60),
 PerformanceEvent(1, 53),
 PerformanceEvent(1, 57),
 PerformanceEvent(1, 60),
 PerformanceEvent(3, 50),
 PerformanceEvent(1, 76),
 PerformanceEvent(3, 25),
 PerformanceEvent(2, 76),
 PerformanceEvent(1, 77),
 PerformanceEvent(3, 25),
 PerformanceEvent(2, 53),
 PerformanceEvent(2, 57),
 PerformanceEvent(2, 60),
 PerformanceEvent(2, 77),
 PerformanceEvent(1, 53),
 PerformanceEvent(1, 57),
 PerformanceEvent(1, 62),
 PerformanceEvent(1, 77),
 PerformanceEvent(3, 75),
 PerformanceEvent(2, 53),
 PerformanceEvent(2, 57),
 PerformanceEvent(2, 62),
 PerformanceEvent(1, 52),
 Performance

In [261]:
note_sequence4 = encoder_pipeline_instance.transform(note_sequence3)[0]
# note_sequence4

#### Determining which position is 1 in the one-hot vector

In [206]:
_event_ranges = [
    (1, 1, 127),
    (2, 1, 127),
    (3, 1, 100)
]

def encode_event(event):
    offset = 0
    for event_type, min_value, max_value in _event_ranges:
        if event[0] == event_type:
            return offset + event[1] - min_value
        offset += max_value - min_value + 1

encode_event((3, 100))

353

#### Investigate time-shifting